# **Notes log:**

**Comments:**
- Missing US Senate Dem results for Walton county
- Many Seminole county results not matching b/w precint- and county-levels

<br>

**Questions:**
- ~~What's our naming convention in cases where candidates are in the same race, for the same party but have the same last name? (...it happened here.) --> *First letter of first name, first two of last name*~~
- ~~What to do for last names with ' ? *Clean these names to remove such characters*~~
-How do I import the pdv functions as a library? *Put in working folder as a .py file, then import*

<br>

**I've almost completely left Peter's code unchanged, except for:**
- Changing "races_list" to "contests_keep"
- Using "P" for primary
- Adding another condition in the candidate name function to capture fringe cases
- Copy-passting in each function, rather than importing pdv script

<br>

**Now need to work on:**
 - Resolving discrepancies b/w precinct and county level results
 - ~~Data checks~~
 - ~~README file~~



In [556]:
import geopandas as gp
import pandas as pd
import os
import numpy as np
import re

#Stop warnings about future changes to libraries from crowding output
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#import pdv_functions as pdv  *Note: Import failing. Come back to this later

#Florida 2022 Election Results Processing#
Precinct-Level Data from [Florida Division of Elections](https://dos.myflorida.com/elections/data-statistics/elections-data/precinct-level-election-results/)

County-Level Data to Run Checks from [Florida Department of State Election Archive](https://results.elections.myflorida.com/Index.asp?ElectionDate=11/8/2022&DATAMODE=)
<br>\----

<br> Code based on [similar processing](https://github.com/nonpartisan-redistricting-datahub/pber_collection/blob/main/FL/2022/fl_2022_results.ipynb) of 2022 FL General Election results by Peter Horton

###Import libraries, load in original precinct-level data, set directories

In [557]:
#Mounting drive to access files
from google.colab import drive
drive.mount('/content/drive')

#Set the working directory
project_folder = '/content/drive/MyDrive/Projects/florida_primary2022'
os.chdir(project_folder)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [558]:
#This Data is from the FL department of state, and can only be downloaded county by county
all_files = os.listdir('/content/drive/MyDrive/Projects/florida_primary2022/raw-from-source/2022-pri-outputofficial')

#Looping through files to import, excluding recounts
li = []
for i in all_files:
    if i not in [".DS_Store",
                 'CLA_PctResults20220823_Recount.txt',
                 'HIL_PctResults20220823_Recount.txt',
                 'NAS_PctResults20220823_Recount.txt',
                 'ORA_PctResults20220823_Recount.txt']:
        ref = './raw-from-source/2022-pri-outputofficial/'
        file_ref = ref+i
        file_prev = pd.read_csv(file_ref,sep="\t",engine='python',index_col=None, header=None, dtype = str)
        li.append(file_prev)
frame = pd.concat(li, axis=0, ignore_index=True)
print(frame.shape)

'\n#This Data is from the FL department of state, and can only be downloaded county by county\nall_files = os.listdir(\'/content/drive/MyDrive/Projects/florida_primary2022/raw-from-source/2022-pri-outputofficial\')\n\n#Looping through files to import, excluding recounts\nli = []\nfor i in all_files:\n    if i not in [".DS_Store",\n                 \'CLA_PctResults20220823_Recount.txt\',\n                 \'HIL_PctResults20220823_Recount.txt\',\n                 \'NAS_PctResults20220823_Recount.txt\',\n                 \'ORA_PctResults20220823_Recount.txt\']:\n        ref = \'./raw-from-source/2022-pri-outputofficial/\'\n        file_ref = ref+i\n        file_prev = pd.read_csv(file_ref,sep="\t",engine=\'python\',index_col=None, header=None, dtype = str)\n        li.append(file_prev)\nframe = pd.concat(li, axis=0, ignore_index=True)\nprint(frame.shape)\n'

In [559]:
#CODE FOR SAVING DATAFRAME TO EASILY LOADABLE PICKLE FILE
#USEFUL FOR AVOIDING LONG LOAD TIME WHILE WORKING WITH THIESE FILES
#frame.to_pickle('./output/frame.pkl')

#LOADING IN THAT .PKL FILE
#frame = pd.read_pickle("./output/frame.pkl")

"\nframe.to_pickle('./output/frame.pkl')\n"

##Process original precinct-level results##

###Clean results before data pivot###

In [561]:
#Rename the columns in the file

col_rename_dict = {0: 'County Code (Three-character abbreviation)',
 1: 'County Name',
 2: 'Election Number',
 3: 'Election Date',
 4: 'Election Name',
 5: 'Unique Precinct Identifier',
 6: 'Precinct Polling Location',
 7: 'Total Registered Voters',
 8: 'Total Registered Republicans',
 9: 'Total Registered Democrats',
 10: 'Total Registered All Other Parties',
 11: 'Contest Name',
 12: 'District',
 13: 'Contest Code (Florida’s 6 digit contest codes)',
 14: 'Candidate/Retention/IssueName/WriteInsCast/OverVotes/UnderVotes',
 15: 'Candidate Party (abbreviation)',
 16: 'Candidate Florida Voter Registration System ID Number',
 17: 'DOE Assigned Candidate Number or Retention/Issue Number',
 18: 'Vote Total'}

In [562]:
#Rename the columns and clean the dataframe
frame.rename(columns = col_rename_dict, inplace = True)
frame['Vote Total'] = frame['Vote Total'].astype(int)

#Set the Unique Precinct Identifier to the Precinct Polling Location, where the Unique Identifier is "N/A"
frame["Unique Precinct Identifier"] = np.where(frame["Unique Precinct Identifier"].isna(), frame["Precinct Polling Location"], frame["Unique Precinct Identifier"])

In [563]:
display(frame.head())
frame['Contest Name'].unique()

,County Code (Three-character abbreviation),County Name,Election Number,Election Date,Election Name,Unique Precinct Identifier,Precinct Polling Location,Total Registered Voters,Total Registered Republicans,Total Registered Democrats,Total Registered All Other Parties,Contest Name,District,Contest Code (Florida’s 6 digit contest codes),Candidate/Retention/IssueName/WriteInsCast/OverVotes/UnderVotes,Candidate Party (abbreviation),Candidate Florida Voter Registration System ID Number,DOE Assigned Candidate Number or Retention/Issue Number,Vote Total
0,ALA,Alachua,NaN,08/23/2022,2022 Primary Election,1,First Baptist Church of Waldo,1509,0,520,0,United States Senator,,120002,Ricardo De La Fuente,DEM,0,82512,11
1,ALA,Alachua,NaN,08/23/2022,2022 Primary Election,2,LaCrosse Town Hall,2462,0,988,0,United States Senator,,120002,Ricardo De La Fuente,DEM,0,82512,12
2,ALA,Alachua,NaN,08/23/2022,2022 Primary Election,3,Legacy Park,3972,0,1419,0,United States Senator,,120002,Ricardo De La Fuente,DEM,0,82512,11
3,ALA,Alachua,NaN,08/23/2022,2022 Primary Election,4,Alachua County Agr. and Equestrian Center,3530,0,1063,0,United States Senator,,120002,Ricardo De La Fuente,DEM,0,82512,14
4,ALA,Alachua,NaN,08/23/2022,2022 Primary Election,5,First Lutheran Church,3190,0,1680,0,United States Senator,,120002,Ricardo De La Fuente,DEM,0,82512,12


array(['United States Senator', 'Representative in Congress', 'Governor',
       'Attorney General', 'Commissioner of Agriculture',
       'State Representative', 'County Commissioner', 'Circuit Judge',
       'School Board', 'State Senator', 'County Court Judge',
       'Charlotte County Airport Authority',
       'Hendry County Hospital Authority', 'Supervisor of Elections',
       'North Lake County Hospital District',
       'Clerk of the Circuit Court and Comptroller', 'Property Appraiser',
       'Ocean Highway and Port Authority', 'County Mayor',
       'Board of County Commissioners', 'Port of Palm Beach',
       'Indian Trail Improvement District',
       'Sarasota County Charter Review Board',
       'Sarasota County Public Hospital Board', 'County Council'],
      dtype=object)

In [564]:
#Helpful for making informed assessment of which races to include in file
'''
unique_precincts = list(frame['Unique Precinct Identifier'].unique())
all_contests = sorted(list(frame['Contest Name'].unique()))
len_all_contests = len(all_contests)
listofzeros = [0] * len_all_contests
num_precs = len(unique_precincts)
contests_dict = dict(zip(all_contests,listofzeros))
in_all_precincts = []
for i in unique_precincts:
    sub_df = frame[frame['Unique Precinct Identifier']==i]
    contests = list(sub_df['Contest Name'].unique())
    for contest in contests:
        contests_dict[contest] = int(contests_dict.get(contest))+1
contests_keep_guess =[]
for k,v in contests_dict.items():
    if v>200:
        print(k,'\t',v)
        contests_keep_guess.append(k)
print(contests_keep_guess)
'''

"\nunique_precincts = list(frame['Unique Precinct Identifier'].unique())\nall_contests = sorted(list(frame['Contest Name'].unique()))\nlen_all_contests = len(all_contests)\nlistofzeros = [0] * len_all_contests\nnum_precs = len(unique_precincts)\ncontests_dict = dict(zip(all_contests,listofzeros))\nin_all_precincts = []\nfor i in unique_precincts:\n    sub_df = frame[frame['Unique Precinct Identifier']==i]\n    contests = list(sub_df['Contest Name'].unique())\n    for contest in contests:\n        contests_dict[contest] = int(contests_dict.get(contest))+1\ncontests_keep_guess =[]\nfor k,v in contests_dict.items():\n    if v>200:\n        print(k,'\t',v)\n        contests_keep_guess.append(k)\nprint(contests_keep_guess)\n"

In [565]:
contests_keep = ['United States Senator', 'Representative in Congress', 'Governor','Attorney General',
                 'Commissioner of Agriculture','State Representative', 'State Senator']
print(contests_keep)

['United States Senator', 'Representative in Congress', 'Governor', 'Attorney General', 'Commissioner of Agriculture', 'State Representative', 'State Senator']


In [566]:
#Filter to only include results for contest types with statewide reach
frame = frame[frame['Contest Name'].isin(contests_keep)]

#Filter out the OverVotes and UnderVotes
filtered_frame = frame[~frame['Candidate/Retention/IssueName/WriteInsCast/OverVotes/UnderVotes'].isin(['OverVotes', 'UnderVotes'])]

#Clean up the precinct column
filtered_frame['Precinct Polling Location'] = filtered_frame['Precinct Polling Location'].astype(str)

#Make the precinct column at least 4 digits
filtered_frame["modified_pre"] = filtered_frame["Unique Precinct Identifier"].astype(str).str.zfill(4)

#Make a column with the 3 letter county code and the precincts
filtered_frame["pct_std"]=filtered_frame['County Code (Three-character abbreviation)']+"-"+filtered_frame["modified_pre"]

filtered_frame.head()

<ipython-input-566-7186e6f22536>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame['Precinct Polling Location'] = filtered_frame['Precinct Polling Location'].astype(str)
<ipython-input-566-7186e6f22536>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["modified_pre"] = filtered_frame["Unique Precinct Identifier"].astype(str).str.zfill(4)
<ipython-input-566-7186e6f22536>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

,County Code (Three-character abbreviation),County Name,Election Number,Election Date,Election Name,Unique Precinct Identifier,Precinct Polling Location,Total Registered Voters,Total Registered Republicans,Total Registered Democrats,...,Contest Name,District,Contest Code (Florida’s 6 digit contest codes),Candidate/Retention/IssueName/WriteInsCast/OverVotes/UnderVotes,Candidate Party (abbreviation),Candidate Florida Voter Registration System ID Number,DOE Assigned Candidate Number or Retention/Issue Number,Vote Total,modified_pre,pct_std
0,ALA,Alachua,NaN,08/23/2022,2022 Primary Election,1,First Baptist Church of Waldo,1509,0,520,...,United States Senator,,120002,Ricardo De La Fuente,DEM,0,82512,11,0001,ALA-0001
1,ALA,Alachua,NaN,08/23/2022,2022 Primary Election,2,LaCrosse Town Hall,2462,0,988,...,United States Senator,,120002,Ricardo De La Fuente,DEM,0,82512,12,0002,ALA-0002
2,ALA,Alachua,NaN,08/23/2022,2022 Primary Election,3,Legacy Park,3972,0,1419,...,United States Senator,,120002,Ricardo De La Fuente,DEM,0,82512,11,0003,ALA-0003
3,ALA,Alachua,NaN,08/23/2022,2022 Primary Election,4,Alachua County Agr. and Equestrian Center,3530,0,1063,...,United States Senator,,120002,Ricardo De La Fuente,DEM,0,82512,14,0004,ALA-0004
4,ALA,Alachua,NaN,08/23/2022,2022 Primary Election,5,First Lutheran Church,3190,0,1680,...,United States Senator,,120002,Ricardo De La Fuente,DEM,0,82512,12,0005,ALA-0005


In [567]:
# Cast the contest name and code columns to string to make the pivot col
filtered_frame["Contest Name"] = filtered_frame["Contest Name"].astype(str)
filtered_frame["Contest Code (Florida’s 6 digit contest codes)"] = filtered_frame["Candidate/Retention/IssueName/WriteInsCast/OverVotes/UnderVotes"].astype(str)

# Fill in the blank candidate party with "WRI"
filtered_frame['Candidate Party (abbreviation)'] = filtered_frame['Candidate Party (abbreviation)'].map({' ':'WRI'}).fillna(filtered_frame['Candidate Party (abbreviation)'])

# Create the pivot column with the contest name and the candidate
filtered_frame["pivot_col"] = filtered_frame["Contest Name"] + "-:-" + filtered_frame["Contest Code (Florida’s 6 digit contest codes)"] + "-:-" + "PARTY:" +filtered_frame['Candidate Party (abbreviation)']

# Where it exists, add in the district to the pivot column
filtered_frame["pivot_col"] = np.where(filtered_frame["District"]!=" ",filtered_frame["pivot_col"] + "-:-" + filtered_frame["District"], filtered_frame["pivot_col"])

#Removing all of the specific party registration data, as it looks like the state doesn't have that
for val in ['Total Registered Republicans','Total Registered Democrats', 'Total Registered All Other Parties']:
    print(filtered_frame[val].unique())

filtered_frame.drop(['Election Number', 'Election Date', 'Total Registered Republicans','Total Registered Democrats', 'Total Registered All Other Parties','Candidate Florida Voter Registration System ID Number','DOE Assigned Candidate Number or Retention/Issue Number'], axis = 1, inplace = True)

<ipython-input-567-3ffb999258d8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["Contest Name"] = filtered_frame["Contest Name"].astype(str)
<ipython-input-567-3ffb999258d8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["Contest Code (Florida’s 6 digit contest codes)"] = filtered_frame["Candidate/Retention/IssueName/WriteInsCast/OverVotes/UnderVotes"].astype(str)
<ipython-input-567-3ffb999258d8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

['0' '667' '971' ... '1384' '2068' '2386']
['520' '988' '1419' ... '6202' '3210' '2242']
['0']


<ipython-input-567-3ffb999258d8>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame.drop(['Election Number', 'Election Date', 'Total Registered Republicans','Total Registered Democrats', 'Total Registered All Other Parties','Candidate Florida Voter Registration System ID Number','DOE Assigned Candidate Number or Retention/Issue Number'], axis = 1, inplace = True)


In [568]:
#Checking how many write-in candidates there are
print(filtered_frame[filtered_frame['Candidate Party (abbreviation)'] == 'WRI']['Candidate/Retention/IssueName/WriteInsCast/OverVotes/UnderVotes'].unique())

#Checking how many parties overall
filtered_frame['Candidate Party (abbreviation)'].unique()

[]


array(['DEM', 'REP'], dtype=object)

###Data pivot and subsequent cleaning

In [569]:
#Pivot the data so that each row has all the results from that precinct
pivoted_2022 = pd.pivot_table(filtered_frame, values=['Vote Total'], index=["pct_std",'County Code (Three-character abbreviation)','County Name',"Unique Precinct Identifier","Precinct Polling Location"],columns=['pivot_col'],aggfunc=sum)

#Clean up the indices
pivoted_2022.reset_index(inplace=True,drop=False)
pivoted_2022.columns = [' '.join(col).strip() for col in pivoted_2022.columns.values]
pivoted_2022.columns = pivoted_2022.columns.str.replace("Vote Total ","")
pivoted_2022 = pivoted_2022.fillna(0)

#Remove single quotes from all column names, part of cleaning candidate surnames like O'Brien
pivoted_2022.columns = [col.replace("'", "") for col in pivoted_2022.columns]

pivoted_2022.head()

,pct_std,County Code (Three-character abbreviation),County Name,Unique Precinct Identifier,Precinct Polling Location,Attorney General-:-Aramis Ayala-:-PARTY:DEM,Attorney General-:-Daniel Uhlfelder-:-PARTY:DEM,Attorney General-:-Jim Lewis-:-PARTY:DEM,Commissioner of Agriculture-:-J. R. Gaillot-:-PARTY:DEM,Commissioner of Agriculture-:-James W. Shaw-:-PARTY:REP,...,State Senator-:-Richard Paul Dembinsky-:-PARTY:DEM-:- District 8,"State Senator-:-Shevrin ""Shev"" Jones-:-PARTY:DEM-:- District 34",State Senator-:-Steve Byers-:-PARTY:REP-:- District 26,State Senator-:-Tracie Davis-:-PARTY:DEM-:- District 5,State Senator-:-Travis Hutson-:-PARTY:REP-:- District 7,State Senator-:-William Wheelen-:-PARTY:REP-:- District 26,United States Senator-:-Brian Rush-:-PARTY:DEM,United States Senator-:-Ricardo De La Fuente-:-PARTY:DEM,United States Senator-:-Val Demings-:-PARTY:DEM,United States Senator-:-William Sanchez-:-PARTY:DEM
0,ALA-0001,ALA,Alachua,1,First Baptist Church of Waldo,54.0,49.0,51.0,41.0,87.0,...,0.0,0.0,0.0,0.0,0.0,0.0,20.0,11.0,120.0,10.0
1,ALA-0002,ALA,Alachua,2,LaCrosse Town Hall,186.0,60.0,90.0,98.0,99.0,...,0.0,0.0,0.0,0.0,0.0,0.0,52.0,12.0,263.0,22.0
2,ALA-0003,ALA,Alachua,3,Legacy Park,198.0,133.0,93.0,114.0,184.0,...,0.0,0.0,0.0,0.0,0.0,0.0,44.0,11.0,352.0,40.0
3,ALA-0004,ALA,Alachua,4,Alachua County Agr. and Equestrian Center,149.0,86.0,80.0,88.0,267.0,...,0.0,0.0,0.0,0.0,0.0,0.0,50.0,14.0,240.0,29.0
4,ALA-0005,ALA,Alachua,5,First Lutheran Church,213.0,210.0,51.0,157.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18.0,12.0,439.0,36.0


In [570]:
#Rename columns
keep_names = ['pct_std', 'County Code (Three-character abbreviation)', 'County Name',
       'Unique Precinct Identifier', 'Precinct Polling Location']

###Loading helper functions for column renaming and dictionary creation###

In [571]:
#Setting up a pre-fab function for assigning legislative abbreviations
#Part of pdv_functions??
def get_level_dist(column_name):
    zfill_level = 2
    if "Representative in Congress" in column_name:
        level = "CON"
    elif "State Senator" in column_name:
        level = "SU"
    elif "State Representative" in column_name:
        level = "SL"
        zfill_level = 3
    else:
        raise ValueError
    return_val = re.findall("District \S*",column_name)
    if (len(return_val)!=0):
        dist = return_val[0].split(" ")[1]
        dist = dist.zfill(zfill_level)
    else:
        raise ValueError
    return level,dist

In [572]:
#Set up contest level
def get_race(contest):
    if "President" in contest:
        level = "PRE"
    elif ("Representative in Congress" in contest or "State Senator" in contest or "State Representative" in contest):
        contest_info = get_level_dist(contest)
        level = contest_info[0]+contest_info[1]
    else:
        print(contest)
        raise ValueError
    return level

In [573]:
#Set up election type (general or primary)
def get_election_type(contest):
    return "P"

In [574]:
#Set up contest three-letter descriptor
def get_race(contest):
    mod_level = ""
    level = contest.split("-:-")[0]
    level_change_dict = {
        'Attorney General':'ATG',
        'Court of Appeals Judge':'CAJ',
        'Governor':'GOV',
        'State Controller':'CNT',
        'STATE QUESTION':'SQ',
        'Secretary of State':'SOS',
        'State Representative':'SL',
        'State Senator':'SU',
        'President':'PRE',
        'United States Senator':'USS',
        'Amendment':'A',
        'State Treasurer':'TRE',
        'Retention of':'SCJ',
        'Representative in Congress':'CON',
        'Chief Financial Officer':'CFO',
        'Commissioner of Agriculture':'COA'}
    for val in level_change_dict.keys():
        if val in level:
            mod_level = level_change_dict[val]
            break
    if mod_level == "":
        print("NO CONTEST", contest)
    if mod_level == 'A':
        mod_level += level.split(":")[0].split(" ")[-1]

    return mod_level



In [575]:
#Set up party single-letter identifier
def get_party(contest):
    if "Amendment" in contest:
        return ""
    elif "Retention of" in contest:
        return contest.split("-:-")[1][0].upper()
    elif "PARTY:DEM" in contest:
        return "D"
    elif "PARTY:REP" in contest:
        return "R"
    elif "PARTY:LPF" in contest:
        return "L"
    ## Reform -> F
    elif "PARTY:REF" in contest:
        return "O"
    elif "PARTY:PSL" in contest:
        return "S"
    elif "PARTY:GRE" in contest:
        return "G"
    elif "PARTY:CPF" in contest:
        return "C"
    elif "PARTY:WRI" in contest:
        return "O"
    elif "PARTY:NPA" or "PARTY:NOP" in contest:
        return "N"
    else:
        print(contest)
        return ValueError


In [576]:
#Set up function for first three letters of candidate's surname
def get_name(contest):
    if "No for Rejection" in contest:
        return "NO"
    elif "Yes for Approval" in contest:
        return "YES"

    if "Retention of" in contest:
        candidate = contest.split("-:-")[0]
        return candidate.upper().split(" ")[-1][0:3]
    else:
        candidate = contest.split("-:-")[1]
        if candidate == "None Of These Candidates":
            return "WRI"
        candidate = candidate.upper()
        if "Governor" in contest:
            likely_last = candidate.split(" ")[0]
        else:
            likely_last = candidate.split(" ")[-1]
        if likely_last in ["JR","III","II","SR"] or (len(candidate.split(" "))==3 and "." not in candidate and '\"' not in candidate):
            return candidate.split(" ")[-2][0:3]
        else:
            return likely_last[0:3]

In [577]:
#Set up function for getting district string
def get_district(contest):
    district_string = contest.split("-:-")[3]
    district_string = district_string.strip()
    likely = district_string.split(" ")[1]
    if "Representative in Congress" in contest or 'State Senator' in contest:
        level = 2
    elif 'State Representative' in contest:
        level = 3
    return likely.zfill(level)


###Apply functions, dictionary export to README, more cleaning###

In [ ]:
#Applying functions to all columns except for ones in keep_names list
contest_name_change_dict = {}

duplicate_value_list = []
for contest in pivoted_2022.columns:
    if contest not in keep_names:
        # Add in a condition about the 20
#         print("E-type", get_election_type(contest))
#         print("Race", get_race(contest))
#         print("Party", get_party(contest))
#         print("Name", get_name(contest))
        if "Representative in Congress" in contest or 'State Representative' in contest or 'State Senator' in contest:
            value = get_election_type(contest) + get_race(contest) + get_district(contest) + get_party(contest) + get_name(contest)
        else:
            value = get_election_type(contest) + "22" + get_race(contest) + get_party(contest) + get_name(contest)
        if value in contest_name_change_dict.values():
            duplicate_value_list.append(value)
        print(len(value))
        contest_name_change_dict[contest] = value


In [579]:
#Checking for duplicate columns
if len(duplicate_value_list) > 0:
    print("DUPLICATE VALUES")
    for val in [(k,v) for k, v in contest_name_change_dict.items() if v in duplicate_value_list]:
        print(val)

DUPLICATE VALUES
('Representative in Congress-:-Alan Grayson-:-PARTY:DEM-:- District 10', 'PCON10DGRA')
('Representative in Congress-:-Terence R. Gray-:-PARTY:DEM-:- District 10', 'PCON10DGRA')


In [580]:
#Clean up edge cases
contest_name_change_dict['Representative in Congress-:-Alan Grayson-:-PARTY:DEM-:- District 10'] = 'PCON10DAGR'
contest_name_change_dict['Representative in Congress-:-Terence R. Gray-:-PARTY:DEM-:- District 10'] = 'PCON10DTGR'

In [581]:
# Create a sorted dictionary to sort the columns of the final dataframe
sorted_contest_name_change_dict = dict(sorted(contest_name_change_dict.items(), key=lambda x:x[1]))

# Export the candidate name change dictionary for the README
holder = pd.DataFrame(sorted_contest_name_change_dict.values(),sorted_contest_name_change_dict.keys())
holder.to_csv("./field_names.csv", index = True)

In [582]:
#Rename the dataframe using this dictionary
pivoted_2022.rename(columns = contest_name_change_dict, inplace = True)

#Define a reversed version of the dictionary
contest_name_change_dict_rev = {v:k for k,v in contest_name_change_dict.items()}

In [583]:
#Examining list of columns

for column in pivoted_2022.columns:
    print(column)

pct_std
County Code (Three-character abbreviation)
County Name
Unique Precinct Identifier
Precinct Polling Location
P22ATGDAYA
P22ATGDUHL
P22ATGDLEW
P22COADGAI
P22COARSHA
P22COADEST
P22COADMOR
P22COARSIM
P22GOVDCAD
P22GOVDCHA
P22GOVDNIC
P22GOVDROB
PCON04RBEA
PCON09RMOR
PCON07DKRU
PCON10DAGR
PCON15DCOH
PCON07DPAS
PCON23DELL
PCON13RMAK
PCON27DMON
PCON20DOMP
PCON13RPAU
PCON27DTAD
PCON04DHIL
PCON07RSAB
PCON07RAL
PCON23DSOR
PCON07RDUK
PCON21RMAS
PCON12RPER
PCON19RDON
PCON10RWIM
PCON25RSPA
PCON28RGIM
PCON28RGAR
PCON22RLAW
PCON15DRAM
PCON06RDAV
PCON12RLEI
PCON13RQUI
PCON14DBRA
PCON23RMCL
PCON10DBRO
PCON07RMIL
PCON20DHOL
PCON22RFRA
PCON08DDOD
PCON11RWEB
PCON03DHAW
PCON23RCER
PCON26RAQU
PCON25DWAS
PCON22RADE
PCON15RGRI
PCON15DGEL
PCON18RTAR
PCON04RAGU
PCON07RBEN
PCON27RPOL
PCON24DWIL
PCON15DBRO
PCON11RSOR
PCON01RMER
PCON12RMIC
PCON23DHOL
PCON07DFER
PCON23RWEI
PCON10DACH
PCON12RMAR
PCON15RTOL
PCON23RPRU
PCON14RJUD
PCON23DMOS
PCON21RBUO
PCON10DBOO
PCON18RRAY
PCON14RTOR
PCON24RNAV
PCON19RHUF
PCON0

In [584]:
#Add a COUNTYFP column
FIPS_dict = {'Alachua': '001',
 'Baker': '003',
 'Bay': '005',
 'Bradford': '007',
 'Brevard': '009',
 'Broward': '011',
 'Calhoun': '013',
 'Charlotte': '015',
 'Citrus': '017',
 'Clay': '019',
 'Collier': '021',
 'Columbia': '023',
 'Desoto': '027',
 'Dixie': '029',
 'Duval': '031',
 'Escambia': '033',
 'Flagler': '035',
 'Franklin': '037',
 'Gadsden': '039',
 'Gilchrist': '041',
 'Glades': '043',
 'Gulf': '045',
 'Hamilton': '047',
 'Hardee': '049',
 'Hendry': '051',
 'Hernando': '053',
 'Highlands': '055',
 'Hillsborough': '057',
 'Holmes': '059',
 'Indian River': '061',
 'Jackson': '063',
 'Jefferson': '065',
 'Lafayette': '067',
 'Lake': '069',
 'Lee': '071',
 'Leon': '073',
 'Levy': '075',
 'Liberty': '077',
 'Madison': '079',
 'Manatee': '081',
 'Marion': '083',
 'Martin': '085',
 'Miami-Dade': '086',
 'Monroe': '087',
 'Nassau': '089',
 'Okaloosa': '091',
 'Okeechobee': '093',
 'Orange': '095',
 'Osceola': '097',
 'Palm Beach': '099',
 'Pasco': '101',
 'Pinellas': '103',
 'Polk': '105',
 'Putnam': '107',
 'St. Johns': '109',
 'St. Lucie': '111',
 'Santa Rosa': '113',
 'Sarasota': '115',
 'Seminole': '117',
 'Sumter': '119',
 'Suwannee': '121',
 'Taylor': '123',
 'Union': '125',
 'Volusia': '127',
 'Wakulla': '129',
 'Walton': '131',
 'Washington': '133'}

pivoted_2022["COUNTYFP"] = pivoted_2022["County Name"].map(FIPS_dict).fillna(pivoted_2022["County Name"])
print(pivoted_2022["COUNTYFP"].unique())

['001' '003' '005' '007' '009' '011' '013' '015' '017' '019' '021' '023'
 '086' '027' '029' '031' '033' '035' '037' '039' '041' '043' '045' '047'
 '049' '051' '053' '055' '057' '059' '061' '063' '065' '067' '069' '071'
 '073' '075' '077' '079' '081' '087' '083' '085' '089' '091' '093' '095'
 '097' '099' '101' '103' '105' '107' '113' '115' '117' '109' '111' '119'
 '121' '123' '125' '127' '129' '131' '133']


In [585]:
#Additional cleaning
pivoted_2022.rename(columns = {"pct_std":"UNIQUE_ID"}, inplace = True)
pivoted_2022[pivoted_2022["UNIQUE_ID"].str.contains("nan")]
for col in list(contest_name_change_dict.values()):
    pivoted_2022[col] = pivoted_2022[col].astype(int)

In [ ]:
pivoted_2022.head()

##Checks against FL Dept. of State's county-level data##

###Read in files, clean



In [587]:
#Read in alternative data source, in this case FL Dept. of State
#Has data aggregated by county
alt_source = pd.read_csv("./raw-from-source/08232022Election.txt", encoding = "latin-1", sep="\t")

#Choosing which contests to look at
contests_keep_alt = ['United States Senator', 'Representative in Congress', 'Governor','Attorney General',
                 'Commissioner of Agriculture','State Representative', 'State Senator']

alt_source = alt_source[alt_source["OfficeDesc"].isin(contests_keep_alt)]

#alt_source.head()

In [588]:
#Add in the District Number where relevant
alt_source["OfficeDesc_Detailed"] = alt_source.apply(lambda x: x["OfficeDesc"] + " "+str(int(x["Juris1num"])) if not pd.isna(x["Juris1num"]) else x["OfficeDesc"], axis = 1)

#Combine the office, candidate first name and candidate last name, clean where all of that info isn't available
alt_source["cand_col"] = alt_source["OfficeDesc_Detailed"] + "-:-" + alt_source["CanNameLast"] + ", " + alt_source["CanNameFirst"] + "-:-" + alt_source["PartyCode"]
alt_source.loc[alt_source["CanNameFirst"].isna(),"cand_col"] = alt_source["OfficeDesc_Detailed"] + "-:-" + alt_source["CanNameLast"]  + "-:-" + alt_source["PartyCode"]

###Pivot, more cleaning

In [589]:
#Pivoting the data, reseting indices, and filling nulls
pivoted_alt = pd.pivot_table(alt_source, index = "CountyName", columns = "cand_col", values = "CanVotes", aggfunc = sum)
pivoted_alt.reset_index(inplace = True, drop = False)
pivoted_alt = pivoted_alt.fillna(0)

#Remove single quotes from all column names, part of cleaning candidate surnames like O'Brien
pivoted_alt.columns = [col.replace("'", "") for col in pivoted_alt.columns]

###Modifying helper functions as needed###

In [652]:
#Functions to set up column names

def get_election_type(contest):
    return "P"


def get_race(contest):
    mod_level = ""
    level = contest.split("-:-")[0]
    level_change_dict = {
        'Attorney General':'ATG',
        'Court of Appeals Judge':'CAJ',
        'Lieutenant Governor':'LTG',
        'Governor':'GOV',
        'State Controller':'CNT',
        'STATE QUESTION':'SQ',
        'Secretary of State':'SOS',
        'State Representative':'SL',
        'State Senator':'SU',
        'President':'PRE',
        'United States Senator':'USS',
        'Amendment':'A',
        'State Treasurer':'TRE',
        'retained in Office':'SCJ',
        'United States Representative':'CON',
        'Chief Financial Officer':'CFO',
        'Commissioner of Agriculture':'COA'}
    for val in level_change_dict.keys():
        if val in level:
            mod_level = level_change_dict[val]
            break
    if mod_level == "":
        print("NO CONTEST", contest)
    if mod_level == 'A':
        mod_level += level.split(":")[0].split(" ")[-1]

    return mod_level

def get_party(contest):
    contest = contest.split("-:-")[2]
    if "Amendment" in contest:
        return ""
    elif "Retention of" in contest:
        return contest.split("-:-")[1][0].upper()
    elif "DEM" in contest:
        return "D"
    elif "REP" in contest:
        return "R"
    elif "LPF" in contest:
        return "L"
    ## Reform -> F
    elif "REF" in contest:
        return "O"
    elif "PSL" in contest:
        return "S"
    elif "GRE" in contest:
        return "G"
    elif "CPF" in contest:
        return "C"
    elif "WRI" in contest:
        return "O"
    elif "NPA" or "NOP" in contest:
        return "N"
    else:
        print(contest)
        return ValueError

def get_name(contest):
    if "No for Rejection" in contest:
        return "NO"
    elif "Yes for Approval" in contest:
        return "YES"

    if "Retention of" in contest:
        candidate = contest.split("-:-")[0]
        return candidate.upper().split(" ")[-1][0:3]
    else:
        candidate = contest.split("-:-")[1]
        if candidate == "None Of These Candidates":
            return "WRI"
        candidate = candidate.upper()
#         candidate = candidate.replace(" ", "")
        if "Governor" in contest:
            likely_last = candidate.split(" ")[1]
        else:
            likely_last = candidate.split(" ")[0]
        if likely_last in ["JR","III","II","SR"] or (len(candidate.split(" "))==3 and "." not in candidate and '\"' not in candidate):
            return candidate.split(" ")[-2][0:3]
        else:
            return likely_last[0:3]

def get_district(contest):
    district_string = contest.split("-:-")[0]
    district_string = district_string.strip()
    likely = district_string.split(" ")[-1]
    if 'State Representative' in contest:
        level = 3
    else:
        level = 2

    return likely.zfill(level)

contest_name_change_dict_second = {}

###Apply functions###

In [591]:
#Checking for duplicate column names

duplicate_value_list = []
for contest in pivoted_alt.columns:
    if contest != "CountyName":
        #print(contest)
        #print("E-type", get_election_type(contest))
        #print("Race", get_race(contest))
        #print("Party", get_party(contest))
        #print("Name", get_name(contest))
        if "United States Representative" in contest or 'State Representative' in contest or 'State Senator' in contest:
            value = get_election_type(contest) + get_race(contest) + get_district(contest) + get_party(contest) + get_name(contest)
        else:
            value = get_election_type(contest) + "22" + get_race(contest) + get_party(contest) + get_name(contest)
        if value in contest_name_change_dict_second.values():
            duplicate_value_list.append(value)
        print(len(value))
        contest_name_change_dict_second[contest] = value

if len(duplicate_value_list) > 0:
    print("DUPLICATE VALUES")
    for val in [(k,v) for k, v in contest_name_change_dict_second.items() if v in duplicate_value_list]:
        print(val)

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
10
10
10


In [592]:
#Apply new column names using dictionary just created
pivoted_alt.rename(columns = contest_name_change_dict_second, inplace = True)
contest_name_change_dict_second_rev = {v:k for k,v in contest_name_change_dict_second.items()}

###Comparison to original results

In [593]:
#Checking out a list of contests that appear in alt source but not OG
{i:contest_name_change_dict_second_rev[i] for i in list(pivoted_alt.columns) if i not in list(pivoted_2022.columns) and i !="CountyName"}

{'P22COADBLE': 'Commissioner of Agriculture-:-Blemur, Naomi-:-DEM',
 'PSL106RROS': 'State Representative 106-:-Ross, Douglas-:-REP',
 'PSL106RLYN': 'State Representative 106-:-Sutjapojnukul, Lynn Su-:-REP',
 'PSL119RPOR': 'State Representative 119-:-Porras, Juan-:-REP',
 'PSL120RALL': 'State Representative 120-:-Allen, Robert-:-REP',
 'PSL120DDAN': 'State Representative 120-:-Horton-Diaz, Daniel Dan-:-DEM',
 'PSL120RLOP': 'State Representative 120-:-Lopez, Rhonda-:-REP',
 'PSL120RJAM': 'State Representative 120-:-Mooney, James Jim-:-REP',
 'PSL023RREI': 'State Representative 23-:-Reinhardt, Paul-:-REP',
 'PSL035DTAH': 'State Representative 35-:-Munoz-Chaffin, Tahitiana T-:-DEM',
 'PSL040DDAV': 'State Representative 40-:-Bracy Davis, LaVon-:-DEM',
 'PSL041DSHA': 'State Representative 41-:-Rose, Shaniqua Shan-:-DEM',
 'PSL065RPIT': 'State Representative 65-:-Gonzalez Pittman, Karen-:-REP',
 'PSL069RPET': 'State Representative 69-:-Petty, Megan-:-REP',
 'PSL093DALB': 'State Representative

In [594]:
#Checking out a list of contests that appear in OG source but not alt
ignore_list = ['UNIQUE_ID',
 'County Code (Three-character abbreviation)',
 'County Name','COUNTYFP','pct_std', 'County Code (Three-character abbreviation)', 'County Name',
       'Unique Precinct Identifier', 'Precinct Polling Location']

{i:contest_name_change_dict_rev[i] for i in list(pivoted_2022.columns) if i not in list(pivoted_alt.columns) and i not in ignore_list}

{'P22COADEST': 'Commissioner of Agriculture-:-Naomi Esther Blemur-:-PARTY:DEM',
 'PCON04RBEA': 'Representative in Congress-:-Aaron Bean-:-PARTY:REP-:- District 4',
 'PCON09RMOR': 'Representative in Congress-:-Adianis Morales-:-PARTY:REP-:- District 9',
 'PCON07DKRU': 'Representative in Congress-:-Al Krulick-:-PARTY:DEM-:- District 7',
 'PCON10DAGR': 'Representative in Congress-:-Alan Grayson-:-PARTY:DEM-:- District 10',
 'PCON15DCOH': 'Representative in Congress-:-Alan M. Cohn-:-PARTY:DEM-:- District 15',
 'PCON07DPAS': 'Representative in Congress-:-Allek Pastrana-:-PARTY:DEM-:- District 7',
 'PCON23DELL': 'Representative in Congress-:-Allen Ellison-:-PARTY:DEM-:- District 23',
 'PCON13RMAK': 'Representative in Congress-:-Amanda Makki-:-PARTY:REP-:- District 13',
 'PCON27DMON': 'Representative in Congress-:-Angel Montalvo-:-PARTY:DEM-:- District 27',
 'PCON20DOMP': 'Representative in Congress-:-Anika Omphroy-:-PARTY:DEM-:- District 20',
 'PCON13RPAU': 'Representative in Congress-:-Anna

In [595]:
#Define a list of columns that appear in both dataframes
shared_cols = [i for i in list(pivoted_2022.columns) if i in list(pivoted_alt.columns)]

In [596]:
#Same for unshared columns
unshared_cols = [i for i in list(pivoted_2022.columns) if i not in list(pivoted_alt.columns)]

In [597]:
#Defining statewide_totals_check & county_totals_check function

def statewide_totals_check(partner_df,source_df,column_list):
    """Compares the totals of two election result dataframes at the statewide total level

    Args:
      partner_df: DataFrame of election results we are comparing against
      source_df: DataFrame of election results we are comparing to
      column_list: List of races that there are votes for

    Returns:
      Nothing, only prints out an analysis
    """
    print("***Statewide Totals Check***")
    for race in column_list:
        if (partner_df[race].sum()- source_df[race].sum() != 0):
            print(race+" has a difference of "+str(partner_df[race].sum()-source_df[race].sum())+" votes")
            print("\ORIGINAL: "+str(partner_df[race].sum())+" votes")
            print("\ALT: "+str(source_df[race].sum())+" votes")
        else:
            print(race + " is equal", "\tALT / ORIGINAL: " + str(partner_df[race].sum()))

def county_totals_check(partner_df,source_df,column_list,county_col,full_print=False):
    """Compares the totals of two election result dataframes at the county level

    Args:
      partner_df: DataFrame of election results we are comparing against
      source_df: DataFrame of election results we are comparing to
      column_list: List of races that there are votes for
      county_col: String of the column name that contains county information
      full_print: Boolean specifying whether to print out everything, including counties w/ similarities

    Returns:
      Nothing, only prints out an analysis
    """

    print("***Countywide Totals Check***")
    print("")
    diff_counties=[]
    for race in column_list:
        diff = partner_df.groupby([county_col]).sum()[race]-source_df.groupby([county_col]).sum()[race]
        for val in diff[diff != 0].index.values.tolist():
            if val not in diff_counties:
                diff_counties.append(val)
        if len(diff[diff != 0]!=0):
            print(race + " contains differences in these counties:")
            for val in diff[diff != 0].index.values.tolist():
                county_differences = diff[diff != 0]
                print("\t"+val+" has a difference of "+str(county_differences[val])+" votes")
                print("\t\ORIGINAL: "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
                print("\t\ALT: "+str(source_df.groupby([county_col]).sum().loc[val,race])+" votes")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
        else:
            print(race + " is equal across all counties")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
    if (len(diff_counties)>0):
        print()
        print(diff_counties)



In [598]:
#Calling statewide function
statewide_totals_check(pivoted_2022, pivoted_alt, shared_cols)

***Statewide Totals Check***
P22ATGDAYA has a difference of -15.0 votes
\ORIGINAL: 637841 votes
\ALT: 637856.0 votes
P22ATGDUHL has a difference of -14.0 votes
\ORIGINAL: 399606 votes
\ALT: 399620.0 votes
P22ATGDLEW has a difference of -8.0 votes
\ORIGINAL: 381567 votes
\ALT: 381575.0 votes
P22COADGAI has a difference of -9.0 votes
\ORIGINAL: 268854 votes
\ALT: 268863.0 votes
P22COARSHA has a difference of -14.0 votes
\ORIGINAL: 571943 votes
\ALT: 571957.0 votes
P22COADMOR has a difference of -12.0 votes
\ORIGINAL: 420597 votes
\ALT: 420609.0 votes
P22COARSIM has a difference of -26.0 votes
\ORIGINAL: 1048749 votes
\ALT: 1048775.0 votes
P22GOVDCAD has a difference of -1.0 votes
\ORIGINAL: 38197 votes
\ALT: 38198.0 votes
P22GOVDCHA has a difference of -25.0 votes
\ORIGINAL: 904499 votes
\ALT: 904524.0 votes
P22GOVDNIC has a difference of -8.0 votes
\ORIGINAL: 535472 votes
\ALT: 535480.0 votes
P22GOVDROB has a difference of -3.0 votes
\ORIGINAL: 36783 votes
\ALT: 36786.0 votes
PSL120DGEN

In [599]:
#Minor renaming for the county-level check
pivoted_alt.rename(columns = {"CountyName":"County Name"}, inplace = True)

In [600]:
#Calling county function
county_totals_check(pivoted_2022, pivoted_alt, shared_cols, "County Name")

***Countywide Totals Check***

P22ATGDAYA contains differences in these counties:
	Seminole has a difference of -15.0 votes
	\ORIGINAL: 16321 votes
	\ALT: 16336.0 votes
P22ATGDUHL contains differences in these counties:
	Seminole has a difference of -14.0 votes
	\ORIGINAL: 10378 votes
	\ALT: 10392.0 votes
P22ATGDLEW contains differences in these counties:
	Seminole has a difference of -8.0 votes
	\ORIGINAL: 5829 votes
	\ALT: 5837.0 votes
P22COADGAI contains differences in these counties:
	Seminole has a difference of -9.0 votes
	\ORIGINAL: 7239 votes
	\ALT: 7248.0 votes
P22COARSHA contains differences in these counties:
	Seminole has a difference of -14.0 votes
	\ORIGINAL: 12378 votes
	\ALT: 12392.0 votes
P22COADMOR contains differences in these counties:
	Seminole has a difference of -12.0 votes
	\ORIGINAL: 9587 votes
	\ALT: 9599.0 votes
P22COARSIM contains differences in these counties:
	Seminole has a difference of -26.0 votes
	\ORIGINAL: 25643 votes
	\ALT: 25669.0 votes
P22GOVDCAD 

##Checks against OpenElections precinct-level data
Especially paying attention to Seminole, Volusia and Walton counties

###Read in file, clean

In [601]:
#Read in alternative data source, in this case OpenElections
#Has precinct-level data
alt_source2 = pd.read_csv("./raw-from-source/20220823__fl__primary__precinct_OpenElections.csv")

#Choosing which contests to look at
contests_keep_alt2 = ['U.S. Senate', 'U.S. House', 'Governor','Attorney General', 'Commissioner of Agriculture','State House', 'State Senate']

alt_source2 = alt_source2[alt_source2["office"].isin(contests_keep_alt2)]

#for i in alt_source2.office.unique():
  #print(i)

In [602]:
#Splitting precinct strings
alt_source2['precinct'] = alt_source2['precinct'].str.replace('PRE ', '')
alt_source2[['precinct', 'poll_loc']] = alt_source2['precinct'].str.split(' |:', n=1, expand=True)

#Removing leading zeroes
alt_source2['precinct'] = alt_source2['precinct'].str.lstrip('0')

#Splitting candidate first and last names
alt_source2[['cand_first', 'cand_last']] = alt_source2['candidate'].str.split(' ', n=1, expand=True)
alt_source2.cand_last = alt_source2.cand_last.str.replace(r'[A-Z]\. ', '',regex=True)

#Add in the District Number where relevant
alt_source2["office_detailed"] = alt_source2.apply(lambda x: x["office"] + " "+str(int(x["district"])) if not pd.isna(x["district"]) else x["office"], axis = 1)

#Combine the office, candidate first name and candidate last name, clean where all of that info isn't available
alt_source2["cand_col"] = alt_source2["office_detailed"] + "-:-" + alt_source2["cand_last"] + ", " + alt_source2["cand_first"] + "-:-" + alt_source2["party"]
alt_source2.loc[alt_source2["cand_first"].isna(),"cand_col"] = alt_source2["office_detailed"] + "-:-" + alt_source2["cand_last"]  + "-:-" + alt_source2["party"]

alt_source2.head()

,county,precinct,office,district,party,candidate,votes,poll_loc,cand_first,cand_last,office_detailed,cand_col
0,Alachua,1,U.S. Senate,NaN,DEM,Ricardo De La Fuente,11,First Baptist Church of Waldo,Ricardo,De La Fuente,U.S. Senate,"U.S. Senate-:-De La Fuente, Ricardo-:-DEM"
1,Alachua,2,U.S. Senate,NaN,DEM,Ricardo De La Fuente,12,LaCrosse Town Hall,Ricardo,De La Fuente,U.S. Senate,"U.S. Senate-:-De La Fuente, Ricardo-:-DEM"
2,Alachua,3,U.S. Senate,NaN,DEM,Ricardo De La Fuente,11,Legacy Park,Ricardo,De La Fuente,U.S. Senate,"U.S. Senate-:-De La Fuente, Ricardo-:-DEM"
3,Alachua,4,U.S. Senate,NaN,DEM,Ricardo De La Fuente,14,Alachua County Agr. and Equestrian Center,Ricardo,De La Fuente,U.S. Senate,"U.S. Senate-:-De La Fuente, Ricardo-:-DEM"
4,Alachua,5,U.S. Senate,NaN,DEM,Ricardo De La Fuente,12,First Lutheran Church,Ricardo,De La Fuente,U.S. Senate,"U.S. Senate-:-De La Fuente, Ricardo-:-DEM"


###Pivot, more cleaning

In [603]:
#Pivoting the data, reseting indices, and filling nulls
pivoted_alt2 = pd.pivot_table(alt_source2, index = "county", columns = "cand_col", values = "votes", aggfunc = sum)
pivoted_alt2.reset_index(inplace = True, drop = False)
pivoted_alt2 = pivoted_alt2.fillna(0)

#Remove single quotes from all column names, part of cleaning candidate surnames like O'Brien
pivoted_alt2.columns = [col.replace("'", "") for col in pivoted_alt2.columns]

pivoted_alt2.head()

,county,"Attorney General-:-Ayala, Aramis-:-DEM","Attorney General-:-Lewis, Jim-:-DEM","Attorney General-:-Uhlfelder, Daniel-:-DEM","Commissioner of Agriculture-:-Esther Blemur, Naomi-:-DEM","Commissioner of Agriculture-:-Gaillot, J.-:-DEM","Commissioner of Agriculture-:-Morales, Ryan-:-DEM","Commissioner of Agriculture-:-Shaw, James-:-REP","Commissioner of Agriculture-:-Simpson, Wilton-:-REP","Governor-:-""Nikki"" Fried, Nicole-:-DEM",...,"U.S. House 8-:-Dodge, Danelle-:-DEM","U.S. House 8-:-Terry, Joanne-:-DEM","U.S. House 9-:-Castillo, Jose-:-REP","U.S. House 9-:-Moore, Scotty-:-REP","U.S. House 9-:-Morales, Adianis-:-REP","U.S. House 9-:-Ortiz, Sergio-:-REP","U.S. Senate-:-De La Fuente, Ricardo-:-DEM","U.S. Senate-:-Demings, Val-:-DEM","U.S. Senate-:-Rush, Brian-:-DEM","U.S. Senate-:-Sanchez, William-:-DEM"
0,Alachua,14545.0,4755.0,8391.0,11514.0,8261.0,7253.0,4815.0,10634.0,14733.0,...,0.0,0.0,0.0,0.0,0.0,0.0,857.0,24002.0,2336.0,2057.0
1,Baker,240.0,500.0,186.0,367.0,210.0,341.0,1765.0,2452.0,340.0,...,0.0,0.0,0.0,0.0,0.0,0.0,57.0,516.0,225.0,176.0
2,Bay,2518.0,2218.0,1658.0,2919.0,1369.0,1998.0,6404.0,13796.0,3080.0,...,0.0,0.0,0.0,0.0,0.0,0.0,339.0,4565.0,997.0,640.0
3,Bradford,412.0,710.0,221.0,581.0,333.0,411.0,1375.0,2218.0,485.0,...,0.0,0.0,0.0,0.0,0.0,0.0,58.0,910.0,276.0,132.0
4,Brevard,20121.0,11100.0,12502.0,18810.0,9795.0,14276.0,17921.0,41905.0,17247.0,...,20350.0,23216.0,0.0,0.0,0.0,0.0,951.0,41956.0,1987.0,1886.0


###Modifying helper functions as needed###

In [604]:
#Updated helper functions to match naming convention of OpenElections file

def get_race2(contest):
    mod_level = ""
    level = contest.split("-:-")[0]
    level_change_dict = {
        'Attorney General':'ATG',
        'Court of Appeals Judge':'CAJ',
        'Lieutenant Governor':'LTG',
        'Governor':'GOV',
        'State Controller':'CNT',
        'STATE QUESTION':'SQ',
        'Secretary of State':'SOS',
        'State House':'SL',
        'State Senate':'SU',
        'President':'PRE',
        'U.S. Senate':'USS',
        'Amendment':'A',
        'State Treasurer':'TRE',
        'retained in Office':'SCJ',
        'U.S. House':'CON',
        'Chief Financial Officer':'CFO',
        'Commissioner of Agriculture':'COA'}
    for val in level_change_dict.keys():
        if val in level:
            mod_level = level_change_dict[val]
            break
    if mod_level == "":
        print("NO CONTEST", contest)
    if mod_level == 'A':
        mod_level += level.split(":")[0].split(" ")[-1]

    return mod_level

def get_district2(contest):
    district_string = contest.split("-:-")[0]
    district_string = district_string.strip()
    likely = district_string.split(" ")[-1]
    if 'State House' in contest:
        level = 3
    else:
        level = 2

    return likely.zfill(level)

contest_name_change_dict_third = {}

###Apply functions###

In [605]:
duplicate_value_list = []
for contest in pivoted_alt2.columns:
    if contest != "county":
        #print(contest)
        #print("E-type", get_election_type(contest))
        #print("Race", get_race(contest))
        #print("Party", get_party(contest))
        #print("Name", get_name(contest))
        if "U.S. House" in contest or 'State House' in contest or 'State Senate' in contest:
            value = get_election_type(contest) + get_race2(contest) + get_district2(contest) + get_party(contest) + get_name(contest)
        else:
            value = get_election_type(contest) + "22" + get_race(contest) + get_party(contest) + get_name(contest)
        if value in contest_name_change_dict_third.values():
            duplicate_value_list.append(value)
        print(len(value))
        contest_name_change_dict_third[contest] = value

if len(duplicate_value_list) > 0:
    print("DUPLICATE VALUES")
    for val in [(k,v) for k, v in contest_name_change_dict_third.items() if v in duplicate_value_list]:
        print(val)

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10

In [606]:
#Clean up edge cases (not sure what's going on w/ this no contest thing)
contest_name_change_dict_third['Representative in Congress-:-Alan Grayson-:-PARTY:DEM-:- District 10'] = 'PCON10DAGR'
contest_name_change_dict_third['Representative in Congress-:-Terence R. Gray-:-PARTY:DEM-:- District 10'] = 'PCON10DTGR'
contest_name_change_dict_third['U.S. Senate-:-De La Fuente, Ricardo-:-DEM'] = 'P22USSDDEL'
contest_name_change_dict_third['U.S. Senate-:-Demings, Val-:-DEM'] = 'P22USSDDEM'
contest_name_change_dict_third['U.S. Senate-:-Rush, Brian-:-DEM'] = 'P22USSDRUS'
contest_name_change_dict_third['U.S. Senate-:-Sanchez, William-:-DEM'] = 'P22USSDSAN'

In [607]:
#Apply new column names using dictionary just created
pivoted_alt2.rename(columns = contest_name_change_dict_third, inplace = True)
contest_name_change_dict_third_rev = {v:k for k,v in contest_name_change_dict_third.items()}

In [608]:
#Not sure why I'm getting duplicates, but just going to drop those for now
#drop_list = ['P22ATGDAYA', 'P22ATGDLEW', 'P22ATGDUHL', 'P22COADBLE', 'P22COADMOR', 'P22COARSIM', 'P22GOVDCHA', 'P22GOVDCAD']
#pivoted_alt2 = pivoted_alt2.drop(columns = drop_list)

pivoted_alt2.columns


Index(['county', 'P22ATGDAYA', 'P22ATGDLEW', 'P22ATGDUHL', 'P22COADBLE',
       'P22COADGAI', 'P22COADMOR', 'P22COARSHA', 'P22COARSIM', 'P22GOVDFRI',
       ...
       'PCON08DDOD', 'PCON08DTER', 'PCON09RCAS', 'PCON09RMOO', 'PCON09RMOR',
       'PCON09RORT', 'P22USSDDEL', 'P22USSDDEM', 'P22USSDRUS', 'P22USSDSAN'],
      dtype='object', length=355)

###Comparisons across all 3 sources for pertinent counties

In [609]:
#Define a list of columns that appear in both dataframes
shared_cols2 = [i for i in list(pivoted_2022.columns) if i in list(pivoted_alt2.columns)]
shared_cols3 = [i for i in list(pivoted_alt.columns) if i in list(pivoted_alt2.columns)]

In [610]:
#Filtering to three counties of note
three_counties = ['Seminole', 'Volusia', 'Walton']
pivoted_2022_check3 = pivoted_2022[pivoted_2022["County Name"].isin(three_counties)]
pivoted_alt_check3 = pivoted_alt[pivoted_alt["County Name"].isin(three_counties)]
pivoted_alt2_check3 = pivoted_alt2[pivoted_alt2["county"].isin(three_counties)]

#Re-index after filter
pivoted_alt2_check3.reset_index(inplace = True, drop = False)
pivoted_2022_check3.reset_index(inplace = True, drop = False)

#Minor renaming for the county-level check
pivoted_alt2_check3.rename(columns = {"county":"County Name"}, inplace = True)

pivoted_alt.iloc[0]

<ipython-input-610-2b24a55941f7>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivoted_alt2_check3.rename(columns = {"county":"County Name"}, inplace = True)


County Name    Alachua
P22ATGDAYA     14545.0
P22ATGDLEW      4755.0
P22ATGDUHL      8391.0
P22COADBLE     11514.0
                ...   
PSU08DWIL          0.0
P22USSDDE        857.0
P22USSDDEM     24002.0
P22USSDRUS      2336.0
P22USSDSAN      2057.0
Name: 0, Length: 220, dtype: object

In [ ]:
#Calling county function, ONLY FOR THREE COUNTIES I WANT
county_totals_check(pivoted_2022_check3, pivoted_alt2_check3, shared_cols2, "County Name")

In [ ]:
#Checking alts against each other
county_totals_check(pivoted_alt_check3, pivoted_alt2_check3, shared_cols3, "County Name")

##Additional checks, minor cleaning

In [642]:
#Looking for nonphysical precincts
#These would require vote allocation
nonphysical_prec = pivoted_2022[pivoted_2022.POLL_LOC.str.contains("Early|VBM")]
#nonphysical_prec

In [643]:
#Check vote totals in nonphysical precincts
names = ['UNIQUE_ID','COUNTYFP','CNTY_CODE','CNTY_NAME','PREC_ID','POLL_LOC']

#Visually inspect
'''
for col in nonphysical_prec.columns:
  if col not in names:
    print(nonphysical_prec[col].unique())
'''

'\nfor col in nonphysical_prec.columns: \n  if col not in names:\n    print(nonphysical_prec[col].unique())\n'

In [644]:
#Vote totals are zero. Removing these non-vote, non-physical precincts
pivoted_2022 = pivoted_2022[~pivoted_2022['POLL_LOC'].str.contains('Early|VBM|Election Day')]

#Not sure what exactly to do with that Election Day precinct in Seminole county either??
#Dropping it for now

In [645]:
#Find the number of zero vote precincts
check_zero_votes = pivoted_2022
check_zero_votes['Sum'] = check_zero_votes.loc[:, ~check_zero_votes.columns.isin(names)].sum(axis=1)
check_zero_votes = check_zero_votes[check_zero_votes['Sum']==0]

#Preview
#pd.set_option('display.max_rows', 220)
display(check_zero_votes)
#pd.set_option('display.max_rows', 5)

<ipython-input-645-a4417bc4f8f3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check_zero_votes['Sum'] = check_zero_votes.loc[:, ~check_zero_votes.columns.isin(names)].sum(axis=1)


,UNIQUE_ID,COUNTYFP,CNTY_CODE,CNTY_NAME,PREC_ID,POLL_LOC,P22ATGDAYA,P22ATGDLEW,P22ATGDUHL,P22COADEST,...,PSU22RGRU,PSU22RJOH,PSU26RBYE,PSU26RWHE,PSU34DESC,PSU34DIGH,PSU34DJON,PSU35DBOO,PSU35DSHA,Sum
27,ALA-0027,001,ALA,Alachua,27,Alachua County SOE,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52,ALA-0051,001,ALA,Alachua,51,Alachua County SOE,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5204,POL-0127,105,POL,Polk,127,Precinct 127,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5621,SEM-2002,117,SEM,Seminole,2002,Election Day,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##Finalize file order and export

In [646]:
pivoted_2022.rename(columns = {'Unique Precinct Identifier':'PREC_ID',
                               'County Name':'CNTY_NAME',
                              'County Code (Three-character abbreviation)':'CNTY_CODE',
                              'Precinct Polling Location':'POLL_LOC'}, inplace = True)

<ipython-input-646-765fe23093a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivoted_2022.rename(columns = {'Unique Precinct Identifier':'PREC_ID',


In [647]:
pivoted_2022 = pivoted_2022[['UNIQUE_ID', 'COUNTYFP', 'CNTY_CODE', 'CNTY_NAME', 'PREC_ID',
       'POLL_LOC']+list(sorted_contest_name_change_dict.values())]

In [648]:
pivoted_2022["UNIQUE_ID"] = np.where(pivoted_2022["UNIQUE_ID"]=="WAS-0001", pivoted_2022["UNIQUE_ID"]+"-"+pivoted_2022["POLL_LOC"],pivoted_2022["UNIQUE_ID"])

In [649]:
pivoted_2022.loc[pivoted_2022["POLL_LOC"]=="nan","POLL_LOC"] = "No Location Provided"

In [650]:
if not os.path.exists("./output/fl_2022_pri_prec/"):
    os.mkdir("./output/fl_2022_pri_prec/")

pivoted_2022.to_csv("./output/fl_2022_pri_prec/fl_2022_pri_prec.csv", index = False)